# Transfer Learning - TP Applicatif : Yolo live avec une webcam

Dans ce TP, notre objectif est d'utiliser l'algorithme yolo avec différents models préentraînés afin de faire de la reconnaissance d'objets en temps réel. Pour ce faire, nous allons prendre en main la librairie ultralytics et l'associer à la bibliothèque cv2 afin d'utiliser le flux live de la caméra pour reconnaître des entités parmi des catégories comprises dans des modèles différents.

> Découpage des utilisations :
## 1 - Modèle yolo V8
Ce modèle comporte 80 catégories générales, allant de *person* à *toothbrush*. La précision s'en retrouve assez haute car l'algorithme peut difficilement confondre les objets de forme claire. Par exemple, un simple dessin au stylo d'une plante en pot sera reconnu facilement. En revanche, des objets de forme peu classiques ont du mal à être reconnus. Une bouteille au design minimal peut donc être confondu avec une télécommande ou un téléphone.

Le code suivant se charge d'importer les librairies nécessaires au bon fonctionnement de la webcam et de l'algorithme YOLO :


In [1]:
from ultralytics import YOLO
import cv2
import math

Ensuite, nous définissons notre modèle, et les classnames (catégories) que notre algorithme va reconnaître :

In [3]:
# model
model = YOLO("yolo-Weights/yolov8n.pt")

classNames = {0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mouse',
 65: 'remote',
 66: 'keyboard',
 67: 'cell phone',
 68: 'microwave',
 69: 'oven',
 70: 'toaster',
 71: 'sink',
 72: 'refrigerator',
 73: 'book',
 74: 'clock',
 75: 'vase',
 76: 'scissors',
 77: 'teddy bear',
 78: 'hair drier',
 79: 'toothbrush'}

Enfin, on lance le flux de la caméra et on lance la boucle de détection principale :

In [4]:
# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 0), 2)

            # confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls] + " : " + str(confidence), org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 4 persons, 188.3ms
Confidence ---> 0.89
Class name --> person
Confidence ---> 0.67
Class name --> person
Confidence ---> 0.61
Class name --> person
Confidence ---> 0.54
Class name --> person
Speed: 0.0ms preprocess, 188.3ms inference, 18.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 persons, 1 tie, 1 laptop, 85.9ms
Confidence ---> 0.87
Class name --> person
Confidence ---> 0.73
Class name --> person
Confidence ---> 0.67
Class name --> person
Confidence ---> 0.61
Class name --> person
Confidence ---> 0.6
Class name --> laptop
Confidence ---> 0.5
Class name --> person
Confidence ---> 0.48
Class name --> tie
Confidence ---> 0.3
Class name --> person
Confidence ---> 0.29
Class name --> person
Speed: 0.0ms preprocess, 85.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 persons, 1 chair, 1 laptop, 86.4ms
Confidence ---> 0.89
Class name --> person
Confidence ---> 0.76
Class name --> person
Confidence ---> 0.73
Class name --> la

KeyboardInterrupt: 

Le résultat attendu avec un exemple dessiné à la main devrait être similaire à ceci :
![potted_plant.png](CR_images/potted_plant.png)

## 2 - Modèle YOLO v8n-oiv7
Ce modèle comporte 601 catégories générales, allant de *Accordion* à *Zucchini*. La précision est moins bonne qu'avec le modèle précédent en raison des nombreuses catégories parfois assez proches (Man / Woman par exemple).

Le code suivant reprend les étapes vues ci-dessus en adaptant au nouveau modèle :


In [5]:
# model
model = YOLO("yolo-Weights/yolov8n-oiv7.pt")

classNames = ['Accordion', 'Adhesive tape', 'Aircraft', 'Airplane', 'Alarm clock', 'Alpaca', 'Ambulance', 'Animal',
              'Ant', 'Antelope', 'Apple', 'Armadillo', 'Artichoke', 'Auto part', 'Axe', 'Backpack', 'Bagel',
              'Baked goods', 'Balance beam', 'Ball', 'Balloon', 'Banana', 'Band-aid', 'Banjo', 'Barge', 'Barrel',
              'Baseball bat', 'Baseball glove', 'Bat (Animal)', 'Bathroom accessory', 'Bathroom cabinet', 'Bathtub',
              'Beaker', 'Bear', 'Bed', 'Bee', 'Beehive', 'Beer', 'Beetle', 'Bell pepper', 'Belt', 'Bench', 'Bicycle',
              'Bicycle helmet', 'Bicycle wheel', 'Bidet', 'Billboard', 'Billiard table', 'Binoculars', 'Bird',
              'Blender', 'Blue jay', 'Boat', 'Bomb', 'Book', 'Bookcase', 'Boot', 'Bottle', 'Bottle opener',
              'Bow and arrow', 'Bowl', 'Bowling equipment', 'Box', 'Boy', 'Brassiere', 'Bread', 'Briefcase', 'Broccoli',
              'Bronze sculpture', 'Brown bear', 'Building', 'Bull', 'Burrito', 'Bus', 'Bust', 'Butterfly', 'Cabbage',
              'Cabinetry', 'Cake', 'Cake stand', 'Calculator', 'Camel', 'Camera', 'Can opener', 'Canary', 'Candle',
              'Candy', 'Cannon', 'Canoe', 'Cantaloupe', 'Car', 'Carnivore', 'Carrot', 'Cart', 'Cassette deck', 'Castle',
              'Cat', 'Cat furniture', 'Caterpillar', 'Cattle', 'Ceiling fan', 'Cello', 'Centipede', 'Chainsaw', 'Chair',
              'Cheese', 'Cheetah', 'Chest of drawers', 'Chicken', 'Chime', 'Chisel', 'Chopsticks', 'Christmas tree',
              'Clock', 'Closet', 'Clothing', 'Coat', 'Cocktail', 'Cocktail shaker', 'Coconut', 'Coffee', 'Coffee cup',
              'Coffee table', 'Coffeemaker', 'Coin', 'Common fig', 'Common sunflower', 'Computer keyboard',
              'Computer monitor', 'Computer mouse', 'Container', 'Convenience store', 'Cookie', 'Cooking spray',
              'Corded phone', 'Cosmetics', 'Couch', 'Countertop', 'Cowboy hat', 'Crab', 'Cream', 'Cricket ball',
              'Crocodile', 'Croissant', 'Crown', 'Crutch', 'Cucumber', 'Cupboard', 'Curtain', 'Cutting board', 'Dagger',
              'Dairy Product', 'Deer', 'Desk', 'Dessert', 'Diaper', 'Dice', 'Digital clock', 'Dinosaur', 'Dishwasher',
              'Dog', 'Dog bed', 'Doll', 'Dolphin', 'Door', 'Door handle', 'Doughnut', 'Dragonfly', 'Drawer', 'Dress',
              'Drill (Tool)', 'Drink', 'Drinking straw', 'Drum', 'Duck', 'Dumbbell', 'Eagle', 'Earrings', 'Egg (Food)',
              'Elephant', 'Envelope', 'Eraser', 'Face powder', 'Facial tissue holder', 'Falcon', 'Fashion accessory',
              'Fast food', 'Fax', 'Fedora', 'Filing cabinet', 'Fire hydrant', 'Fireplace', 'Fish', 'Flag', 'Flashlight',
              'Flower', 'Flowerpot', 'Flute', 'Flying disc', 'Food', 'Food processor', 'Football', 'Football helmet',
              'Footwear', 'Fork', 'Fountain', 'Fox', 'French fries', 'French horn', 'Frog', 'Fruit', 'Frying pan',
              'Furniture', 'Garden Asparagus', 'Gas stove', 'Giraffe', 'Girl', 'Glasses', 'Glove', 'Goat', 'Goggles',
              'Goldfish', 'Golf ball', 'Golf cart', 'Gondola', 'Goose', 'Grape', 'Grapefruit', 'Grinder', 'Guacamole',
              'Guitar', 'Hair dryer', 'Hair spray', 'Hamburger', 'Hammer', 'Hamster', 'Hand dryer', 'Handbag',
              'Handgun', 'Harbor seal', 'Harmonica', 'Harp', 'Harpsichord', 'Hat', 'Headphones', 'Heater', 'Hedgehog',
              'Helicopter', 'Helmet', 'High heels', 'Hiking equipment', 'Hippopotamus', 'Home appliance', 'Honeycomb',
              'Horizontal bar', 'Horse', 'Hot dog', 'House', 'Houseplant', 'Human arm', 'Human beard', 'Human body',
              'Human ear', 'Human eye', 'Human face', 'Human foot', 'Human hair', 'Human hand', 'Human head',
              'Human leg', 'Human mouth', 'Human nose', 'Humidifier', 'Ice cream', 'Indoor rower', 'Infant bed',
              'Insect', 'Invertebrate', 'Ipod', 'Isopod', 'Jacket', 'Jacuzzi', 'Jaguar (Animal)', 'Jeans', 'Jellyfish',
              'Jet ski', 'Jug', 'Juice', 'Kangaroo', 'Kettle', 'Kitchen & dining room table', 'Kitchen appliance',
              'Kitchen knife', 'Kitchen utensil', 'Kitchenware', 'Kite', 'Knife', 'Koala', 'Ladder', 'Ladle', 'Ladybug',
              'Lamp', 'Land vehicle', 'Lantern', 'Laptop', 'Lavender (Plant)', 'Lemon', 'Leopard', 'Light bulb',
              'Light switch', 'Lighthouse', 'Lily', 'Limousine', 'Lion', 'Lipstick', 'Lizard', 'Lobster', 'Loveseat',
              'Luggage and bags', 'Lynx', 'Magpie', 'Mammal', 'Man', 'Mango', 'Maple', 'Maracas',
              'Marine invertebrates', 'Marine mammal', 'Measuring cup', 'Mechanical fan', 'Medical equipment',
              'Microphone', 'Microwave oven', 'Milk', 'Miniskirt', 'Mirror', 'Missile', 'Mixer', 'Mixing bowl',
              'Mobile phone', 'Monkey', 'Moths and butterflies', 'Motorcycle', 'Mouse', 'Muffin', 'Mug', 'Mule',
              'Mushroom', 'Musical instrument', 'Musical keyboard', 'Nail (Construction)', 'Necklace', 'Nightstand',
              'Oboe', 'Office building', 'Office supplies', 'Orange', 'Organ (Musical Instrument)', 'Ostrich', 'Otter',
              'Oven', 'Owl', 'Oyster', 'Paddle', 'Palm tree', 'Pancake', 'Panda', 'Paper cutter', 'Paper towel',
              'Parachute', 'Parking meter', 'Parrot', 'Pasta', 'Pastry', 'Peach', 'Pear', 'Pen', 'Pencil case',
              'Pencil sharpener', 'Penguin', 'Perfume', 'Person', 'Personal care', 'Personal flotation device', 'Piano',
              'Picnic basket', 'Picture frame', 'Pig', 'Pillow', 'Pineapple', 'Pitcher (Container)', 'Pizza',
              'Pizza cutter', 'Plant', 'Plastic bag', 'Plate', 'Platter', 'Plumbing fixture', 'Polar bear',
              'Pomegranate', 'Popcorn', 'Porch', 'Porcupine', 'Poster', 'Potato', 'Power plugs and sockets',
              'Pressure cooker', 'Pretzel', 'Printer', 'Pumpkin', 'Punching bag', 'Rabbit', 'Raccoon', 'Racket',
              'Radish', 'Ratchet (Device)', 'Raven', 'Rays and skates', 'Red panda', 'Refrigerator', 'Remote control',
              'Reptile', 'Rhinoceros', 'Rifle', 'Ring binder', 'Rocket', 'Roller skates', 'Rose', 'Rugby ball', 'Ruler',
              'Salad', 'Salt and pepper shakers', 'Sandal', 'Sandwich', 'Saucer', 'Saxophone', 'Scale', 'Scarf',
              'Scissors', 'Scoreboard', 'Scorpion', 'Screwdriver', 'Sculpture', 'Sea lion', 'Sea turtle', 'Seafood',
              'Seahorse', 'Seat belt', 'Segway', 'Serving tray', 'Sewing machine', 'Shark', 'Sheep', 'Shelf',
              'Shellfish', 'Shirt', 'Shorts', 'Shotgun', 'Shower', 'Shrimp', 'Sink', 'Skateboard', 'Ski', 'Skirt',
              'Skull', 'Skunk', 'Skyscraper', 'Slow cooker', 'Snack', 'Snail', 'Snake', 'Snowboard', 'Snowman',
              'Snowmobile', 'Snowplow', 'Soap dispenser', 'Sock', 'Sofa bed', 'Sombrero', 'Sparrow', 'Spatula',
              'Spice rack', 'Spider', 'Spoon', 'Sports equipment', 'Sports uniform', 'Squash (Plant)', 'Squid',
              'Squirrel', 'Stairs', 'Stapler', 'Starfish', 'Stationary bicycle', 'Stethoscope', 'Stool', 'Stop sign',
              'Strawberry', 'Street light', 'Stretcher', 'Studio couch', 'Submarine', 'Submarine sandwich', 'Suit',
              'Suitcase', 'Sun hat', 'Sunglasses', 'Surfboard', 'Sushi', 'Swan', 'Swim cap', 'Swimming pool',
              'Swimwear', 'Sword', 'Syringe', 'Table', 'Table tennis racket', 'Tablet computer', 'Tableware', 'Taco',
              'Tank', 'Tap', 'Tart', 'Taxi', 'Tea', 'Teapot', 'Teddy bear', 'Telephone', 'Television', 'Tennis ball',
              'Tennis racket', 'Tent', 'Tiara', 'Tick', 'Tie', 'Tiger', 'Tin can', 'Tire', 'Toaster', 'Toilet',
              'Toilet paper', 'Tomato', 'Tool', 'Toothbrush', 'Torch', 'Tortoise', 'Towel', 'Tower', 'Toy',
              'Traffic light', 'Traffic sign', 'Train', 'Training bench', 'Treadmill', 'Tree', 'Tree house', 'Tripod',
              'Trombone', 'Trousers', 'Truck', 'Trumpet', 'Turkey', 'Turtle', 'Umbrella', 'Unicycle', 'Van', 'Vase',
              'Vegetable', 'Vehicle', 'Vehicle registration plate', 'Violin', 'Volleyball (Ball)', 'Waffle',
              'Waffle iron', 'Wall clock', 'Wardrobe', 'Washing machine', 'Waste container', 'Watch', 'Watercraft',
              'Watermelon', 'Weapon', 'Whale', 'Wheel', 'Wheelchair', 'Whisk', 'Whiteboard', 'Willow', 'Window',
              'Window blind', 'Wine', 'Wine glass', 'Wine rack', 'Winter melon', 'Wok', 'Woman', 'Wood-burning stove',
              'Woodpecker', 'Worm', 'Wrench', 'Zebra', 'Zucchini']

# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 0), 2)

            # confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls] + " : " + str(confidence), org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 Clothing, 1 Human face, 1 Man, 151.1ms
Confidence ---> 0.86
Class name --> Human face
Confidence ---> 0.42
Class name --> Clothing
Confidence ---> 0.4
Class name --> Man
Speed: 0.0ms preprocess, 151.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Clothing, 4 Human faces, 2 Mans, 120.2ms
Confidence ---> 0.85
Class name --> Human face
Confidence ---> 0.65
Class name --> Man
Confidence ---> 0.59
Class name --> Human face
Confidence ---> 0.51
Class name --> Clothing
Confidence ---> 0.46
Class name --> Human face
Confidence ---> 0.44
Class name --> Man
Confidence ---> 0.3
Class name --> Human face
Speed: 0.0ms preprocess, 120.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Clothing, 3 Human faces, 2 Mans, 86.4ms
Confidence ---> 0.85
Class name --> Human face
Confidence ---> 0.73
Class name --> Man
Confidence ---> 0.67
Class name --> Human face
Confidence ---> 0.54
Class name --> Clothing
Confidence ---> 0.53

KeyboardInterrupt: 

Ici, le résultat peut être bien plus mitigé, comme avec l'exemple suivant où la catégorie est approximative selon la position du sujet. On peut le voir sur cette image avec la confusion man/woman (les deux sont superposés, le modèle a donc du mal à trancher) :


## 3 - Transfer Learning YOLO V8 :
Ici, nous allons partir de la base du modèle YOLO V8 pour lui faire apprendre seulement 3 nouvelles catégories, celles existantes (les 80) seront remplacé par : *waterbottle*, *can* et *snackbar*.
Le code suivant permet d'utiliser le modèle existant pour effectuer un transfer learning et ajouter les catégories nécessaires :

In [ ]:
# Charger le modèle pré-entrainé
model = YOLO("yolo-Weights/yolov8n.pt")

# Réinitialiser les classes pour correspondre à vos nouvelles classes
model.reset_head(num_classes=3)  # Vos nouvelles classes uniquement

# Lancer l'entraînement avec les nouvelles données
model.train(data="data.yaml", epochs=50, imgsz=640, batch=16)

Attention, vous trouverez ci-joint le fichier data.yaml, qu'il faut modifier à chaque modification du modèle.

Nous utilisons un dataset découpé en 3 catégories : *train*, *valid* et *test* afin d'apprendre à reconnaître les objets visés.Il faut que dans chaque dossiers, il y ai 2 sous dossier, un pour les images et un pour les labels de chaque image. Les fichiers labels contiennent l'indice de la classe correspondante (ici 0 = can, 1 = snackbag et 2 = waterbottle), les coordonnées x et y du centre de l'objet, la largeur et la hauteur de la box de l'objet.

Le résultat se présente sous de forme de deux fichiers : *best.pt* et *last.pt*. En les utilisant comme modèles, on peut désormais reconnaître les trois catégories ajoutées.

Exemple avant utilisation du best.pt :
![img.png](CR_images/img.png)
HERE IMAGE BOITE 2 conserv pas reconnue

![img.png](CR_images/img.png)
HERE IMAGE BOITE 2 conserv reconnue

Le code reste très similaire pour l'utilisation du nouveau modèle :

In [ ]:
# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
model = YOLO("runs/detect/train/weights/best.pt")

classNames = ["can", "snackbag", "waterbottle"]

while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 0), 2)

            # confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls] + " : " + str(confidence), org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()